# Library
---
*Import library* yang diperlukan dalam Proyek Rekomendasi Buku (*Content-Based Filtering*).

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Exploratory Data Analysis (EDA)
---

## Gathering Data
Mengumpulkan data-data yang diperlukan, di dalam proyek ini terdapat tiga buah dataset yang dapat digunakan yaitu:
- Books.csv
- Ratings.csv
- Users.csv

Ketiga dataset tersebut akan disimpan ke dalam tiga variabel berbeda yaitu `books`, `ratings`, dan `users`.

In [ ]:
books = pd.read_csv('/kaggle/input/book-recommendation-dataset/Books.csv')
ratings = pd.read_csv('/kaggle/input/book-recommendation-dataset/Ratings.csv')
users = pd.read_csv('/kaggle/input/book-recommendation-dataset/Users.csv')

Membuat dataframe baru untuk menggabungkan ketiga dataset dengan nama `merged_df`.

In [ ]:
merged_df = pd.merge(ratings, books, on='ISBN')
merged_df = pd.merge(merged_df, users, on='User-ID')
merged_df

Tampilkan lima data paling atas dari ketiga dataset tersebut menggunakan fungsi `head()`.

In [ ]:
books.head()

Setelah menjalankan kode di atas, dapat dilihat bahwa dataset `Books.csv` memiliki kolom:
- ISBN: Kode unik dari sebuah buku
- Book-Title: Judul buku
- Book-Author: Pengarang buku
- Year-Of-Publication: Tahun terbit buku
- Publisher: Penerbit buku
- Image-URL-S: Cover buku berukuran kecil
- Image-URL-M: Cover buku berukuran sedang
- Image-URL-L: Cover buku berukuran besar

In [ ]:
ratings.head()

Setelah menjalankan kode di atas, dapat dilihat bahwa dataset `Ratings.csv` memiliki kolom:
- User-ID: Kode unik dari pengguna yang memberikan penilaian
- ISBN: Kode unik dari sebuah buku
- Book-Rating: Penilaian buku nilai terendah dimulai dari angka 0

In [ ]:
users.head()

Setelah menjalankan kode di atas, dapat dilihat bahwa dataset `Users.csv` memiliki kolom:
- User-ID: Kode unik dari pengguna
- Location: Lokasi pengguna
- Age: Usia pengguna

Hal menarik yang dapat dilihat dari dataset `Users.csv` terdapat banyak *missing value* yang akan ditangani selanjutnya.
Sebelum itu, mari kita lihat jumlah keseluruhan buku, rating, dan user pada proyek kali ini.

In [ ]:
print('Jumlah keseluruhan buku: ', len(books.ISBN.unique()))
print('Jumlah keseluruhan rating: ', len(ratings[['ISBN', 'User-ID']].drop_duplicates()))
print('Jumlah keseluruhan user: ', len(users['User-ID'].unique()))

In [ ]:
print('Shape dataframe buku: ', books.shape)
print('Shape dataframe rating: ', ratings.shape)
print('Shape dataframe user: ', users.shape)

Gunakan fungsi `info()` untuk mengetahui informasi penting seperti tipe data yang digunakan pada proyek kali ini.

In [ ]:
books.info()

Setelah menjalankan kode di atas, dapat dilihat bahwa seluruh kolom pada dataset `Books.csv` memiliki tipe data *object*.
Ada hal unik yang didapati ketika menjalankan kode di atas, dapat dilihat bahwa kolom `Year-Of-Publication` bertipe data *object* sedangkan tahun publikasi umumnya bertipe data integer, oleh karena itu akan dilakukan perbaikan tipe data.

## Mengubah Tipe Data

In [ ]:
## Ketika menjalankan kode di bawah ini muncul ValueError: invalid literal for int() with base 10: 'DK Publishing Inc'
## Artinya value pada `Year-Of-Publication` ada yang bernilai 'DK Publishing Inc'. Sepertinya terdapat kesalahan input.
# books['Year-Of-Publication'].astype('int')

In [ ]:
books[(books['Year-Of-Publication'] == 'DK Publishing Inc') | (books['Year-Of-Publication'] == 'Gallimard')]

Membuang value pada `Year-Of-Publication` ada yang bernilai 'DK Publishing Inc' dan 'Gallimard'.

In [ ]:
temp = (books['Year-Of-Publication'] == 'DK Publishing Inc') | (books['Year-Of-Publication'] == 'Gallimard')
books = books.drop(books[temp].index)
books[(books['Year-Of-Publication'] == 'DK Publishing Inc') | (books['Year-Of-Publication'] == 'Gallimard')]

Mengubah tipe data pada `Year-Of-Publication`.

In [ ]:
books['Year-Of-Publication'] = books['Year-Of-Publication'].astype(int)
print(books.dtypes)

Tipe data pada kolom `Year-Of-Publication` sudah menjadi integer.

In [ ]:
ratings.info()

Setelah menjalankan kode di atas, dapat dilihat bahwa kolom `User-ID` dan `Book-Rating` pada dataset `Ratings.csv` memiliki tipe data *integer* sedangkan kolom `ISBN` bertipe *object*.

In [ ]:
users.info()

Setelah menjalankan kode di atas, dapat dilihat bahwa kolom-kolom pada `Ratings.csv` memiliki tipe data sebagai berikut:
- `User-ID` bertipe integer
- `Location` bertipe object
- `Age` bertipe desimal

## Distribusi Data

In [ ]:
# Grouping'Book-Author' dan hitung jumlah buku yang ditulis oleh masing-masing penulis
author_counts = books.groupby('Book-Author')['Book-Title'].count()
# Urutkan penulis dalam urutan menurun
sorted_authors = author_counts.sort_values(ascending=False)
# Pilih 20 penulis teratas
top_20_authors = sorted_authors.head(20)
# Plot 20 penulis teratas dan buku yang ditulis oleh penulis kemudian dihitung menggunakan plot batang
plt.figure(figsize=(12, 6))
top_20_authors.plot(kind='bar')
plt.xlabel('Nama Penulis')
plt.ylabel('Jumlah Buku')
plt.title('20 Penulis Teratas Berdasarkan Jumlah Buku')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

Setelah menjalankan kode di atas dapat dilihat bahwa Penulis Agatha Christie menulis paling banyak buku yaitu sebanyak > 600 buku.

In [ ]:
# Grouping'Publisher' dan hitung jumlah buku yang dipublikasi oleh masing-masing penerbit
publisher_counts = books.groupby('Publisher')['Publisher'].count()
# Urutkan penerbit dalam urutan menurun
sorted_publisher = publisher_counts.sort_values(ascending=False)
# Pilih 20 penerbit teratas
top_20_publisher = sorted_publisher.head(20)
# Plot 20 penerbit teratas dan buku yang diterbitkan oleh penerbit kemudian dihitung menggunakan plot batang
plt.figure(figsize=(12, 6))
top_20_publisher.plot(kind='bar')
plt.xlabel('Nama Penerbit')
plt.ylabel('Jumlah Buku')
plt.title('20 Penerbit Teratas Berdasarkan Jumlah Buku')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

Setelah menjalankan kode di atas dapat dilihat bahwa Penerbit Harlequin menerbitkan paling banyak buku yaitu sebanyak > 7000 buku.

In [ ]:
# Grouping'Year-Of-Publication' dan hitung jumlah buku yang dipublikasi oleh masing-masing tahun terbit
year_of_publication_counts = books.groupby('Year-Of-Publication')['Year-Of-Publication'].count()
# Urutkan tahun terbit dalam urutan menurun
sorted_year_of_publication = year_of_publication_counts.sort_values(ascending=False)
# Pilih 20 tahun terbit teratas
top_20_year_of_publication = sorted_year_of_publication.head(20)
# Plot 20 tahun terbit teratas dan buku yang diterbitkan oleh penerbit kemudian dihitung menggunakan plot batang
plt.figure(figsize=(12, 6))
top_20_year_of_publication.plot(kind='bar')
plt.xlabel('Tahun Terbit')
plt.ylabel('Jumlah Buku')
plt.title('20 Tahun Terbit Teratas Berdasarkan Jumlah Buku')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

Setelah menjalankan kode di atas dapat dilihat bahwa Tahun Terbit paling banyak menerbitkan buku yaitu pada tahun 2002 sebesar > 17.500 buku.

In [ ]:
plt.figure(figsize=(8, 6))
plt.hist(ratings['Book-Rating'], bins=10, edgecolor='black')
plt.xlabel('Rating Buku (0-10)')
plt.ylabel('Jumlah Buku')
plt.title('Distribusi Rating Buku')
plt.show()

Setelah menjalankan kode di atas dapat dilihat bahwa rating buku sebagian besar mendapaptkan rating 0.

In [ ]:
# Hitung rata-rata rating dari setiap judul buku
mean_rating_by_title = merged_df.groupby('Book-Title')['Book-Rating'].mean()
# Pilih 20 judul buku dengan rata-rata rating terbesar
top_20_titles = mean_rating_by_title.nlargest(20)
top_20_titles

Setelah menjalankan kode di atas dapat dilihat bahwa berikut merupakan 20 judul buku dengan rata-rata rating terbesar.

In [ ]:
# Hitung rata-rata rating dari setiap judul buku
max_rating_by_title = merged_df.groupby('Book-Title')['Book-Rating'].mean()
# Pilih 20 judul buku dengan rata-rata rating terkecil
top_20_titles = max_rating_by_title.nsmallest(20)
top_20_titles

Setelah menjalankan kode di atas dapat dilihat bahwa berikut merupakan 20 judul buku dengan rata-rata rating terkecil.

## Assessing Data
Tahap selanjutnya adalah Penulis akan mengkaji dataset yang digunakan dalam proyek ini dengan mengecek:
1. Apakah ada missing value?
2. Apakah ada duplikasi?

### Apakah ada missing value?

In [ ]:
books.isnull().sum()

In [ ]:
ratings.isnull().sum()

In [ ]:
users.isnull().sum()

Setelah menjalankan kode di atas, dapat dilihat bahwa dataset `Books.csv` memiliki beberapa *missing value*, jumlah *missing value* yang ada pada kolom:
- `Book-Author` berjumlah 1
- `Publisher` berjumlah 2
- `Image-URL-L` berjumlah 3

Karena jumlahnya relatif sedikit maka pada proyek kali ini akan ditangani dengan cara membuang *missing value* tersebut.

### Apakah ada data duplikat?

In [ ]:
books.duplicated().sum()

In [ ]:
ratings.duplicated().sum()

In [ ]:
users.duplicated().sum()

Setelah menjalankan kode di atas, dapat dilihat bahwa tidak ada data duplikasi pada ketiga dataset yang digunakan dalam proyek ini.

## Menangani Missing Value
Menangani missing value pada dataframe `books`. Karena missing value yang ada pada dataframe relatif sedikit maka pada proyek ini Penulis akan mencoba menerapkan teknik drop pada value tersebut.

In [ ]:
books = books.dropna()

In [ ]:
books.isnull().sum()

Karena dataset yang dimiliki sangat banyak dan alokasi memori tidaklah cukup untuk memproses seluruh data, maka di dalam proyek ini Penulis hanya akan mengambil data pertama hingga data ke 20.000 (exclude data ke 20.000).

In [ ]:
books = books[:20000]

In [ ]:
books.shape

In [ ]:
books.describe()

In [ ]:
ratings.describe()

In [ ]:
users.describe()

In [ ]:
books

Selanjutnya, hal yang perlu dilakukan adalah mengonversi data series menjadi list menggunakan library `tolist()` dari numpy.

In [ ]:
isbn = books['ISBN'].tolist()
book_title = books['Book-Title'].tolist()
book_author = books['Book-Author'].tolist()
year_of_publication = books['Year-Of-Publication'].tolist()
publisher = books['Publisher'].tolist()
image_url_s = books['Image-URL-S'].tolist()
image_url_m = books['Image-URL-M'].tolist()
image_url_l = books['Image-URL-L'].tolist()

Tahap berikutnya, hal yang perlu dilakukan adalah membuat *dictionary* untuk menentukan pasangan *key-value* pada dataset books.

In [ ]:
books = pd.DataFrame({
    'isbn': isbn,
    'book_title': book_title,
    'book_author': book_author,
    'year_of_publication': year_of_publication,
    'publisher': publisher,
    'image_url_s': image_url_s,
    'image_url_m': image_url_m,
    'image_url_l': image_url_l,
})
books

# Model Development
---
## TF-IDF Vectorizer
Pada proyek ini, Penulis menggunakan fungsi `tfidfvectorizer()` dari library sklearn.

In [ ]:
# Inisialisasi TfidfVectorizer
tf = TfidfVectorizer()
# Melakukan perhitungan idf pada data `book_author`
tf.fit(books['book_author']) 
# Mapping array dari fitur index integer ke fitur nama
tf.get_feature_names_out() 

Selanjutnya, lakukan fit dan transformasi ke dalam bentuk matriks.

In [ ]:
# Melakukan fit lalu ditransformasikan ke bentuk matrix
tfidf_matrix = tf.fit_transform(books['book_author']) 
# Melihat ukuran matrix tfidf
tfidf_matrix.shape 

Perhatikanlah, matriks yang kita miliki berukuran (20000, 8877). Nilai 20000 merupakan ukuran data dan 8877 merupakan matrik nama penulis buku.

Untuk menghasilkan vektor tf-idf dalam bentuk matriks, kita menggunakan fungsi `todense()`. 

In [ ]:
# Mengubah vektor tf-idf dalam bentuk matriks dengan fungsi todense()
tfidf_matrix.todense()

Selanjutnya mari lihat matriks tf-idf untuk beberapa judul buku dan nama penulis buku.

In [ ]:
pd.DataFrame(
    tfidf_matrix.todense(), 
    columns=tf.get_feature_names_out(),
    index=books.book_title
).sample(10, axis=1,replace=True).sample(10, axis=0)

## Cosine Similarity
Pada tahap sebelumnya, Penulis telah berhasil mengidentifikasi korelasi antara judul buku dan penulis buku. Sekarang, Penulis akan menghitung derajat kesamaan (similarity degree) antar judul buku dengan teknik cosine similarity.

In [ ]:
# Menghitung cosine similarity pada matrix tf-idf
cosine_sim = cosine_similarity(tfidf_matrix) 
cosine_sim

In [ ]:
# Membuat dataframe dari variabel cosine_sim dengan baris dan kolom berupa nama judul buku
cosine_sim_df = pd.DataFrame(cosine_sim, index=books['book_title'], columns=books['book_title'])
print('Shape:', cosine_sim_df.shape)
 
# Melihat similarity matrix pada setiap judul buku
cosine_sim_df.sample(5, axis=1).sample(10, axis=0)

Dengan cosine similarity, Penulis berhasil mengidentifikasi kesamaan antara satu judul buku dengan judul buku lainnya. Shape (20000, 20000) merupakan ukuran matriks similarity dari data yang dimiliki. Berdasarkan data yang ada, matriks di atas sebenarnya berukuran 20000 judul buku x 20000 judul buku (masing-masing dalam sumbu X dan Y). Artinya, Penulis mengidentifikasi tingkat kesamaan pada 20000 nama judul buku. Tapi tentu Penulis tidak bisa menampilkan semuanya. Oleh karena itu, 20000 hanya memilih 10 judul pada baris vertikal dan 5 judul pada sumbu horizontal seperti pada contoh di atas.
Dengan data kesamaan (similarity) judul buku yang diperoleh dari kode sebelumnya, Penulis akan merekomendasikan judul buku yang mirip (similar) dengan judul buku yang sebelumnya pernah dibaca oleh pengguna.

In [ ]:
def content_based_filtering(book_title, top_n=5):
    # Ambil indeks buku di DataFrame
    book_index = books[books['book_title'] == book_title].index[0]
    # Hitung nilai cosine similarity untuk buku tersebut
    similarity_scores = cosine_sim[book_index]
    # Urutkan indeks berdasarkan nilai cosine similarity 
    top_indices = similarity_scores.argsort()[::-1][:top_n+1]
    # Buang input buku yang dimasukan dari rekomendasi
    top_indices = top_indices[top_indices != book_index]
    # Ambil judul buku yang direkomendasikan
    recommended_books = books.loc[top_indices, 'book_title']
    # Hitung nilai kesamaan dari buku-buku tersebut
    similarity_values = similarity_scores[top_indices]
    recommendations_df = pd.DataFrame({'Recommended Books': recommended_books, 'Similarity': similarity_values})
    return recommendations_df

In [ ]:
book_title = "Chicken Soup for the Preteen Soul - 101 Stories of Changes, Choices and Growing Up for Kids, ages 10-13"
top_n = 5
recommendations_df = content_based_filtering(book_title, top_n)
recommendations_df

Sistem berhasil memberikan rekomendasi 5 judul yang mirip dengan input judul buku yang telah dibaca pengguna.